## Anatomy of a NIfTI

teach you using Python and Jupyter notebooks. We'll start with a basic overview of how we can load, view, and play around with MR images using Python's neuroimaging libraries (<code>Nibabel</code>). 

### Intro to NIfTI

NIfTI is one of the most ubiquitous file formats for storing neuroimaging data. We'll cover a few details to get started working with them. If you're interested in learning more about NIfTI images, we highly recommend [this blog post about the NIfTI format](http://brainder.org/2012/09/23/the-nifti-file-format/).

### Reading NIfTI Images

[NiBabel](http://nipy.org/nibabel/) is a Python package for reading and writing neuroimaging data. To learn more about how NiBabel handles NIfTIs, check out the [Working with NIfTI images](http://nipy.org/nibabel/nifti_images.html) page of the NiBabel documentation.

In [1]:
import nibabel as nib

First, use the `load()` function to create a NiBabel image object from a NIfTI file. We'll load in an example T1w image from the ds000030 dataset we just introduced.

In [3]:
t1_img = nib.load('../data/ds000030/sub-10171/anat/sub-10171_T1w.nii.gz')

FileNotFoundError: No such file or no access: '../data/ds000030/sub-10171/anat/sub-10171_T1w.nii.gz'

Loading in a NIfTI file with `nibabel` gives us a special type of data object which encodes all the information in the NIfTI file. Each bit of informationis called an **attribute** in Python's terminology. To see all of these attributes, type `t1_img.` and <kbd>Tab</kbd>.  
There are three main attributes that we'll discuss today:

#### 1. [Header](http://nipy.org/nibabel/nibabel_images.html#the-image-header): contains metadata about the image, such as image dimensions, data type, etc.

In [ ]:
t1_hdr = t1_img.header
print(t1_hdr)

The `t1_img` image header is a Python **dictionary**. Dictionaries are containers that hold pairs of objects - keys and values. Let's take a look at all of the keys.
Similar to `t1_img` in which attributes can be accessed by typing `t1_img.` and hitting <kbd>Tab</kbd>, you can do the same with `t1_hdr`. In particular, we'll be using a **method** belonging to `t1_hdr` that will allow you to view the keys associated with it.

In [ ]:
t1_hdr.keys()

Notice that **methods** require you to include `()` at the end of them whereas **atributes** do not.
The key difference between a method and an attribute is:
- Attributes are *stored values* kept within an object
- Methods are *processes* that we can run using the object. Usually a method takes attributes, performs an operation on them, then returns it for you to use.
You can view the *methods* and *attributes* of a particular object by typing:

~~~python
?t1_img
~~~

The output above is a list of **keys** you can use from `t1_hdr` to access **values**. We can access the value stored by a given key by typing:
|
```python
t1_hdr['<key_name>']
```

<b>EXERCISE:</b> Extract the value of <code>pixdim</code> from <code>t1_hdr</code>

In [ ]:
t1_hdr['pixdim']

#### 2. Data
The data is a multidimensional array representing the image data.

In [ ]:
t1_data = t1_img.get_fdata()
t1_data

In [ ]:
type(t1_data)

Each element in the array corresponds to an intensity value. The range of values typically goes from 0 (black) to 255 (white).

We can check some basic properties of the array.  
The `type()` function will only tell you that a variable is a NumPy array. It won’t tell you the type of data inside of the array.

In [ ]:
t1_data.dtype

This tells us that the NumPy array’s elements are floating-point numbers.   
The data type of an image controls the range of possible intensities. As the number of possible values increases, the amount of space the image takes up in memory also increases.

<b>EXERCISE:</b> How can we see the number of dimensions in the <code>t1_data</code> array? What about the array's shape? Once again, all of the attributes of the array can be seen by typing <code>t1_data.</code> and <kbd>Tab</kbd>.

In [ ]:
t1_data.ndim

In [ ]:
t1_data.shape

The shape of the data always has at least 3 dimensions (X, Y, and Z) and sometimes a 4th, T (time).  
This T1w image has 3 dimensions. This means that the brain was scanned in 176 slices with a resolution of 256 x 256 voxels per slice.

<img src="mri_slices.jpg" alt="Drawing" align="middle" width="300px"/>

Sourced from: Tariq, Humera & Burney, S.M.Aqil. (2014). Brain MRI literature review for interdisciplinary studies. Journal of biomedical graphics and computing. 4. 41-53. 10.5430/jbgc.v4n4p41. 

#### 3. [Affine](http://nipy.org/nibabel/coordinate_systems.html): tells the position of the image array data in a *reference space*

How do we identify the location of an object in space? Coordinate systems! The most common coordinate system in neuroimaging is called *RAS* (right (X), anterior (Y), superior (Z)). A coordinate of (2, -10, 5) refers to a point in space 2 mm to the right, 10 mm to the back and 5 mm above. The affine array helps us identify a real location for each voxel in our image data.

<img src="coordinate_systems.png" alt="Drawing" align="middle" width="300px"/>

Sourced from: https://www.slicer.org/wiki/Coordinate_systems

In [ ]:
t1_affine = t1_img.affine
t1_affine

### Working With Image Data

#### Slicing

n-dimensional images are just stacks of numpy arrays.  Each value in the array is assigned to an x, y or z coordinate.  
<img src="numpy_arrays.png" alt="Drawing" align="middle" width="600px"/>

You'll recall our example T1w image is a 3D image with dimensions $176 \times 256 \times 256$.

Slicing does exactly what it seems to imply. Giving our 3D volume, we pull out a 2D slice of our data. Here's an example of slicing from left to right (**sagittal slicing**):

<img src="https://upload.wikimedia.org/wikipedia/commons/5/56/Parasagittal_MRI_of_human_head_in_patient_with_benign_familial_macrocephaly_prior_to_brain_injury_%28ANIMATED%29.gif"/>

This gif is a series of 2D images or **slices** moving from left to right. We'll now look into how we can pull slices from our 3D image

***
Sourced from: https://en.wikipedia.org/wiki/Neuroimaging#/media/File:Parasagittal_MRI_of_human_head_in_patient_with_benign_familial_macrocephaly_prior_to_brain_injury_(ANIMATED).gif

***

Note that we have two ways of working with neuroimaging data. 


1. We have the actual <code>t1_img</code> we loaded using <code>nibabel</code>
2. We have the array we pulled from <code>t1_img</code> in the form of <code>t1_data</code>


For each of these representations of our data we can pull "slices" using different methods.


First we'll work with the array in the form of <code>t1_img</code>. Since this is a <code>numpy</code> array, we can use the same method as we'd use when dealing with 3D arrays in Python (R and MATLAB are very similar here!).

In [1]:
#Pull the 10th "sagittal slice"
sagittal_slice = t1_data[10,:,:]

Notice the following structure: 

1. We can **index** an array using the array followed by square brackets as follows t1_data[x,y,z]

With our data:
- <code>x</code> is the left to right index
- <code>y</code> is the back to front index
- <code>z</code> is the bottom to top index


So here we're selecting the 10th slice going from left to right (there are 176 slices going from left to right!)

The <code>:</code>, indicates that we want to grab *everything*. 

In plain english we want to:

**Move 10 spots from the left, then grab a full 2D picture here**

<b>EXERCISE:</b> Now try selecting the 70th slice from the back (this is called a **coronal slice**).
It helps to think of this as follows:

Move 70 spots from the back, then grab the full 2D picture here. 

In [ ]:
coronal_slice = t1_data[:,70,:]

Finally try grabbing a **axial slice**, specifically the 126th slice from the bottom:

In [ ]:
axial_slice = t1_data[:,:,126]

As mentioned earlier we can also slice using <code>t1_img</code>, which we loaded through <code>nibabel</code>. It's sometimes nicer to work with this since we don't have to deal with arrays directly like we did with <code>t1_data</code>. 

Slicing with <code>t1_img</code> is just as straightforward and involves using <code>t1_img.slicer[x,y,z]</code>, but with a single caveat which we'll demonstrate here using the 10th slice from the left:

In [ ]:
sagittal_slice = t1_img.slicer[10:11,:,:]

Notice the difference? Here we have to format a sagittal slice as follows:

<code>t1_img.slicer[ (slice) :  (slice# +1) ,:,:]</code>

This is just a result of how <code>slicer</code> is programmed, but everything else remains the same. 

**Exercise**

Use <code>t1_img.slicer</code> to select the:

1. 70th coronal slice from the back
2. 126th axial slice from the bottom

In [2]:
#Coronal slice
coronal_slice = t1_img.slicer[:,70:71,:]

In [3]:
#Axial Slice
axial_slice = t1_img.slicer[:,:,126:127]

We've been slicing and dicing brain images but we have no idea what they look like! In the next section we'll show you how you can visualize brain slices using <code>nibabel</code>!

#### Visualizing
If we wanted to get a quick view of our data, Nibabel provides this functionality through a method called <code>orthoview()</code>. Here's how it looks in action

In [ ]:
import matplotlib.pyplot as plt

### Writing NIfTI Images

Let's save the mask we just created to a file.

In [ ]:
img_mask = nib.Nifti1Image(mask_apply, t1_affine, t1_hdr)
img_mask.to_filename('../data/test_mask.nii.gz')

Notice that *methods* include <code>()</code> at end whereas *attributes* do not. 

The key difference between a method and an attribute is:
- Attributes are *stored values* kept within an object
- Methods are *processes* that we can run using the object. Usually a method takes *attributes*, performs an operation on them, then returns it for you to use.

You can view the *methods* and *attributes* of a particular object by typing:

~~~python
?t1_img
~~~

The output above is a list of **keys** you can use from <code>t1_hdr</code> to access **values**. 

We can access the value stored by a given key by typing:

```python
t1_hdr['<key_name>']
```

<b>EXERCISE:</b> Extract the value of <code>pixdim</code> from <code>t1_hdr</code>

In [ ]:
t1_hdr['pixdim']

## 2. MR Data
As you've seen above, the header contains useful information that gives us information about the properties (metadata) associated with the MR data we've loaded in. Now we'll move in to loading the actual *image data itself*. We can achieve this by using the *method* called <code>t1_img.get_data()</code> 

In [1]:
data = t1_img.get_data()
print(data)

NameError: name 't1_img' is not defined

<code>t1_img.get_data()</code> returns an <code>array</code> object which contains 3 dimensions. You can think of <code>data</code> as a 3D version of a picture (more accurately, a volume):


<img src="numpy_arrays.png" alt="Drawing" align="middle" width="500px"/>

A 3D array contains a value for each (x,y,z) coordinate in our image. You can think of our <code>data</code> object as this 3D volume made up of cubes, and these cubes make up a "picture" of the MR scan as shown below:



<img src="http://www.sprawls.org/mripmt/MRI10/MR10-2.jpg" alt="Drawing" align="middle" width="500px"/>

An analogy that might help you think about this data is that while typical 2D pictures are made out of 2D squares called pixels, a 3D MR image is made up of 3D cubes called voxels.

Now let's inspect some *attributes* of this <code>data</code> object!

First, let's get an idea about how big it is. The <code>.shape</code> attribute tells us this information:

In [ ]:
data.shape

The 3 numbers given here represent the number of values *along a respective dimension (x,y,z)*. That means in total there are:

$$x * y * z = value$$ voxels in total!

How do we examine what value a particular voxel is? We can inspect the value of a voxel by selecting an **index** as follows:

~~~python
data[x,y,z]
~~~

So for example we can inspect a voxel at coordinates (10,20,3) by doing the following:

In [1]:
data[10,20,3]

NameError: name 'data' is not defined

This yields a single value representing the intensity of the signal at a particular voxel! We'll talk about how you can pull not just one voxel but an *array* of voxels for visualization and analysis in just a bit!

## 3. Affine Matrix

The final important piece of metadata associated with an image file is the **affine matrix**. 

To explain this, recall that we referred to coordinates in our data as (x,y,z) coordinates such that:

- x is the first dimension of <code>data</code>
- y is the second dimension of <code>data</code>
- z is the third dimension of <code>data</code>

Although this tells us how to access our data in terms of voxels in a 3D volume, it doesn't tell us much about the actual dimensions in our data (centimetres, right or left, up or down, back or front). The affine matrix allows us to translate between *voxel coordinates (x,y,z)* and world space coordinates in (left/right,bottom/top,back/front). An important thing to note is that in reality in which order you have:

- left/right
- bottom/top
- back/front

Depends on how you've constructed the affine matrix, but for the data we're dealing with it always refers to:

- Right
- Anterior
- Superior

Applying the affine matrix is done through using a *linear map* (matrix multiplication) on voxel coordinates (x,y,z). 

***
The concept of an affine matrix may seem confusing at first but an example might help gain an intuition:

Suppose we have two voxels located in the following locations that we are about:

$$(15,2,90)$$
$$(64,100,2)$$

And we wanted to know what the distances between these two voxels are in terms of real world distances (millimetres). This information cannot be derived from using voxel coordinates so we turn to the **affine matrix**. 

Now, the affine matrix we'll be using happens to be encoded in **RAS**. That means once we apply the matrix our coordinates are as follows:

$$(\text{Right},\text{Anterior},\text{Superior})$$

So increasing a coordinate value in the first dimension corresponds to moving to the right of the person being scanned. 

Applying our affine matrix yields the following coordinates:

$$(90.23,0.2,2.15)$$
$$(10.25,30.5,9.2)$$

This means that:

- Voxel 1 is $90.23-10.25= 79.98$ millimetres to the right
- Voxel 1 is $0.2-30.5= -30.3$ in the A axis. Negative values mean move posterior
- Voxel 1 is $2.15-9.2= -7.05$ in the S axis. Negatve values mean move inferior

***

This covers the basics of how MR NiFTI data and metadata is stored and organized in the context of Python. In the next segment we'll talk a bit about an increasingly important component of MR data analysis - data organization. This is a key component to reproducible analysis and so we'll spend a bit of time here before moving onto actual image manipulation and analysis. 